# Рекомендация тарифов

Цель проекта - построить модель машинного обучения которая подберет подходящий тариф для пользователей мобильной связи.
Представлен датасет с данными клиентов, которые уже перешли на эти тарифы.  
Необходимо построить модель для задачи классификации, которая выберет подходящий тариф. 

## Открытие и изучение файла

In [1]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.dummy import DummyClassifier

In [2]:
df = pd.read_csv('/datasets/users_behavior.csv')

df.info()
display(df.head())
df['is_ultra'].value_counts()
print('Количество пропусков:', '\n', df.isna().sum())
print('Количество дубликатов:', df.duplicated().sum())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


Количество пропусков: 
 calls       0
minutes     0
messages    0
mb_used     0
is_ultra    0
dtype: int64
Количество дубликатов: 0


In [3]:
# Определение корреляции между признаками
df.corr()

,calls,minutes,messages,mb_used,is_ultra
calls,1.000000,0.982083,0.177385,0.286442,0.207122
minutes,0.982083,1.000000,0.173110,0.280967,0.206955
messages,0.177385,0.173110,1.000000,0.195721,0.203830
mb_used,0.286442,0.280967,0.195721,1.000000,0.198568
is_ultra,0.207122,0.206955,0.203830,0.198568,1.000000


In [4]:
# Удаление одного из признаков с сильной корреляцией
df = df.drop(['calls'], axis=1)
df.corr()

,minutes,messages,mb_used,is_ultra
minutes,1.000000,0.173110,0.280967,0.206955
messages,0.173110,1.000000,0.195721,0.203830
mb_used,0.280967,0.195721,1.000000,0.198568
is_ultra,0.206955,0.203830,0.198568,1.000000


### Вывод:
Предотставленный датасет имеет 3214 строки и 5  столбцов. Пропусков, дубликатов нет.  
Целевой признак бинарный и неравномерно распределен.  
У признаков "minutes" и "calls" сильная корреляция - 0.982083, что негативно повлияет на модель обучения.
Признак "calls" был удален.

## Разбивка датасета на выборки

In [5]:
# Разбивка на обучающую выборку, валидационную выборку и тестовую выборку
df_train, df_valid_test = train_test_split(df, test_size=0.4, random_state=12345, stratify=df['is_ultra'])
df_valid, df_test = train_test_split(df_valid_test, test_size=0.5, random_state=6789,
                                     stratify=df_valid_test['is_ultra'])

In [6]:
# Выделение в выборках признаков и целевого признака

features_train = df_train.drop(['is_ultra'], axis=1)
target_train = df_train['is_ultra']

features_valid = df_valid.drop(['is_ultra'], axis=1)
target_valid = df_valid['is_ultra']

features_test = df_test.drop(['is_ultra'], axis=1)
target_test = df_test['is_ultra']

### Итоги:
Выборка разделена на  обучающую выборку (60%), валидационную выборку (20%), тестовую выборку (20%).  
В выборках выделены  признаки "features" и целевой признак "target".

## Исследование моделей

In [7]:
# Выбор наилучшей модели "дерево решений" по метрике "accuracy"
best_result = 0
for i in range(1, 15):
    for k in ['gini', 'entropy']:
        model = DecisionTreeClassifier(random_state=12345, max_depth=i, criterion=k)
        model.fit(features_train, target_train)
        result = model.score(features_valid, target_valid)
        if result > best_result:
            best_tree = model
            best_result = result
            depth = i
            criterion = k
            
print('accuracy:', '%.3f' % best_result, 'max_depth:', depth, 'criterion:', criterion, sep='\n')

accuracy:
0.806
max_depth:
6
criterion:
gini


In [8]:
# Выбор наилучшей модели "случайный лес" по метрике "accuracy"
best_result = 0
for est in range(1, 102, 10):
    for k in ['gini', 'entropy']:
        model = RandomForestClassifier(random_state=12345, n_estimators=est, criterion=k)
        model.fit(features_train, target_train)
        result = model.score(features_valid, target_valid)
        if result > best_result:
            best_forest = model
            best_result = result
            estimators = est
            criterion = k

print(' accuracy:', '%.3f' % best_result, '\n', 'n_estimators:', est, '\n', 'criterion:', criterion)

 accuracy: 0.809 
 n_estimators: 101 
 criterion: gini


In [9]:
# Выбор наилучшей модели "логистической регресиии" по метрике "accuracy"
best_result = 0
for i in ['liblinear', 'lbfgs']:
    model = LogisticRegression(random_state=12345, solver=i)
    model.fit(features_train, target_train)
    result = model.score(features_valid, target_valid)
    if result > best_result:
            best_regress = model
            best_result = result
            solver = i

print(' accuracy:', '%.3f' % best_result, '\n', 'solver:', solver)

 accuracy: 0.706 
 solver: liblinear


## Проверка моделей на тестовой выборке

In [10]:
result_tree = best_tree.score(features_test, target_test)
result_forest = best_forest.score(features_test, target_test)
result_regress = best_regress.score(features_test, target_test)
print(" дерево решений:", '%.3f' % result_tree, '\n', "случайный лес:", '%.3f' % result_forest,
      '\n', "логистическая регресиия:",  '%.3f' % result_regress)

 дерево решений: 0.770 
 случайный лес: 0.810 
 логистическая регресиия: 0.723


### Вывод:
Было проведено исследование моделей "дерево решений", "случайный лес", "логистическая регресиия".  
При обучении моделей менялись значения некторых гиперпараметров.
Наивысшую точность на валидационной выборке показала модель "дерево решений".   
При проверки моделей на тестовой выборке наивысшая точность у модели "случайный лес". Модель "дерево решений" показала меньшую точность из-за эффекта "переобучения"

## Проверка моделей на адекватность

In [11]:
# Проверка модели на адекватность с помощью фиктивного классификатора
strategies = ['most_frequent', 'stratified', 'uniform', 'constant']
test_scores = []
for s in strategies:
    if s =='constant':
        model_d = DummyClassifier(strategy = s, random_state = 12345, constant = 0)
    else:
        model_d = DummyClassifier(strategy = s, random_state = 12345)
    model_d.fit(features_train, target_train)
    score = model_d.score(features_test, target_test)
    test_scores.append(score)
print('Максимальная оценка моделей с фиктивным классификатором:', '%.3f' % max(test_scores))

Максимальная оценка моделей с фиктивным классификатором: 0.694


### Вывод:
Была проведена проверка моделей на адекватность с помощью фиктивного классификатора с различными гиперпараметрами.  
Проверка показала, что модели "дерево решений", "случайный лес", "логистическая регресиия" могут применятся на данном датасете, так как их точность выше чем максимальная точность фиктивного классификатора. 